# Model-based transfer experiments

This notebook contains experiments with model-based transfer techniques.

## Change working directory to project root

In [ ]:
import os
ROOT_DIRECTORIES = {'dogwood', 'tests'}
if set(os.listdir('.')).intersection(ROOT_DIRECTORIES) != ROOT_DIRECTORIES:
    os.chdir('../..')

## Exploration

In [ ]:
from itertools import chain
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from mlops.dataset.versioned_dataset import VersionedDataset
from mlops.model.versioned_model import VersionedModel
from dogwood.pretraining.pretraining_pool import PretrainingPool
from dogwood.model_expansion import expand_dense_layer

In [ ]:
MAX_PIXEL_VALUE = 255
MNIST_IMAGE_SHAPE = (28, 28)

In [ ]:
pool = PretrainingPool()
print('Models:')
for model_path in pool.get_available_models():
    print(model_path)
print()
print('Datasets:')
for dataset_path in pool.get_available_datasets():
    print(dataset_path)

In [ ]:
dataset_imagenet = VersionedDataset(pool.get_dataset_path('imagenet-mini'))
model_vgg16 = VersionedModel(pool.get_model_path('VGG16'))
model_effnet = VersionedModel(pool.get_model_path('EfficientNetB7'))

In [ ]:
model_vgg16.model.summary()

In [ ]:
model_effnet.model.summary()

In [ ]:
# TODO mnist layer expansion, starting from one neuron.
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float32) / MAX_PIXEL_VALUE
X_test = X_test.astype(np.float32) / MAX_PIXEL_VALUE

In [ ]:
model = Sequential(
    [
        Flatten(input_shape=MNIST_IMAGE_SHAPE, name="flatten"),
        Dense(1, activation="relu", name="dense_1"),
        Dense(10, activation="softmax", name="dense_2"),
    ],
    name="micro_mnist_model",
)
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [ ]:
history = model.fit(X_train, y_train, epochs=10)

In [ ]:
max_num_hidden_layer_neurons = 10
num_epochs_per_new_neuron = 2

In [ ]:
model = Sequential(
    [
        Flatten(input_shape=MNIST_IMAGE_SHAPE, name="flatten"),
        Dense(1, activation="relu", name="dense_1"),
        Dense(10, activation="softmax", name="dense_2"),
    ],
    name="micro_mnist_model",
)
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
histories = []
for num_hidden_layer_neurons in range(max_num_hidden_layer_neurons):
    history = model.fit(X_train, y_train, epochs=num_epochs_per_new_neuron)
    histories.append(history)
    if num_hidden_layer_neurons < max_num_hidden_layer_neurons:
        model = expand_dense_layer(model, 'dense_1', 1)
        model.compile(
            optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["sparse_categorical_accuracy"],
        )

In [ ]:
acc_by_epoch = list(chain.from_iterable(
    [history.history['sparse_categorical_accuracy'] for history in histories]))

In [ ]:
plt.plot(range(len(acc_by_epoch)), acc_by_epoch)
plt.xlim([0, len(acc_by_epoch) - 1])
plt.ylim([0, 1])
plt.show()

In [ ]:
model = Sequential(
    [
        Flatten(input_shape=MNIST_IMAGE_SHAPE, name="flatten"),
        Dense(10, activation="relu", name="dense_1"),
        Dense(10, activation="softmax", name="dense_2"),
    ],
    name="micro_mnist_model",
)
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(X_train, y_train, epochs=20)

In [ ]:
plt.plot(
    range(len(history.history['sparse_categorical_accuracy'])),
    history.history['sparse_categorical_accuracy']
)
plt.xlim([0, len(history.history['sparse_categorical_accuracy']) - 1])
plt.ylim([0, 1])
plt.show()